📌실질적으로 돌아가서 결과가 나온 코드블록  
-> 1-2. 카테고리형 변수타입 변환  
-> 4. LGBM 카테고리형 변환 중 4-1, 4-2

In [ ]:
#패키지 불러오기
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, RobustScaler

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, ElasticNet, Ridge
from sklearn.metrics import mean_squared_error , r2_score
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

import lightgbm
from lightgbm import LGBMRegressor, early_stopping
print(lightgbm.__version__)

import hyperopt
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
print(hyperopt.__version__)

3.3.2
0.2.7


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = "/content/drive/MyDrive/Euron/Bus_Tayo/"

# 1. 스케일링 과정

In [ ]:
train = pd.read_csv('train.csv')
test =pd.read_csv('test.csv')

In [ ]:
X = train.drop(labels=['18~20 ride', '18~20 takeoff'],axis=1)
y = train['18~20 ride']
X_test = test.drop(labels=['18~20 ride', '18~20 takeoff'],axis=1)
y_test = test['18~20 ride']

In [ ]:
X

,month,type,route_ID,6~8 ride,8~10 ride,10~12 ride,12~14 ride,14~16 ride,16~18 ride,6~8 takeoff,8~10 takeoff,10~12 takeoff,12~14 takeoff,14~16 takeoff,16~18 takeoff,bus_station_PK
0,202209,0,123000010.0,654,809,576,574,643,651,354,753,769,800,763,991,77100000001
1,202209,0,100100073.0,788,1265,1217,1074,839,731,325,782,696,982,803,950,67100000001
2,202209,0,100100549.0,227,677,282,193,381,558,708,1114,504,450,353,209,31100000002
3,202209,0,100100012.0,89,171,101,76,92,96,653,1070,287,232,159,92,58100000002
4,202209,0,100100009.0,376,759,381,312,344,464,1064,1765,509,460,451,228,24100000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450525,202308,0,100100495.0,0,0,0,0,0,0,0,1,0,0,0,1,81999800002
450526,202308,4,100100391.0,2,2,0,1,1,1,7,0,1,1,1,3,1999800003
450527,202308,4,100100391.0,0,0,0,0,0,0,1,3,1,2,5,3,60999800004
450528,202308,4,100100400.0,0,0,0,0,0,0,2,11,3,7,2,11,78999800005


## 1-1. 원핫인코딩 사용

In [ ]:
def oh_scale(scaler, X, X_test):
  if scaler == 'minmax':
    transformer = MinMaxScaler()
  elif scaler == 'standard':
    transformer = StandardScaler()
  elif scaler == 'robust':
    transformer = RobustScaler()

  X_scaled1 = transformer.fit_transform(X[['month', '6~8 ride', '8~10 ride', '10~12 ride', '12~14 ride', '14~16 ride',
                                         '16~18 ride', '6~8 takeoff', '8~10 takeoff', '10~12 takeoff',
                                         '12~14 takeoff', '14~16 takeoff', '16~18 takeoff']])
  X_scaled2 = X
  X_scaled2[['month', '6~8 ride', '8~10 ride', '10~12 ride', '12~14 ride', '14~16 ride',
                                         '16~18 ride', '6~8 takeoff', '8~10 takeoff', '10~12 takeoff',
                                         '12~14 takeoff', '14~16 takeoff', '16~18 takeoff']] = X_scaled1


  X_scaled3 = transformer.transform(X_test[['month', '6~8 ride', '8~10 ride', '10~12 ride', '12~14 ride', '14~16 ride',
                                         '16~18 ride', '6~8 takeoff', '8~10 takeoff', '10~12 takeoff',
                                         '12~14 takeoff', '14~16 takeoff', '16~18 takeoff']])
  X_scaled4 = X_test
  X_scaled4[['month', '6~8 ride', '8~10 ride', '10~12 ride', '12~14 ride', '14~16 ride',
                                         '16~18 ride', '6~8 takeoff', '8~10 takeoff', '10~12 takeoff',
                                         '12~14 takeoff', '14~16 takeoff', '16~18 takeoff']] = X_scaled3

  X_scaled = pd.get_dummies(X_scaled2, columns = ['bus_station_PK','type'])
  X_test_scaled = pd.get_dummies(X_scaled4, columns = ['bus_station_PK','type'])
  return X_scaled, X_test_scaled

In [ ]:
X_minmax, X_test_minmax = oh_scale('minmax', X, X_test)
X_standard, X_test_standard = oh_scale('standard', X, X_test)
X_robust, X_test_robust = oh_scale('robust', X, X_test)

## 1-2. 카테고리형 변수타입 변환

In [ ]:
def cat_scale(scaler, X, X_test):
  if scaler == 'minmax':
    transformer = MinMaxScaler()
  elif scaler == 'standard':
    transformer = StandardScaler()
  elif scaler == 'robust':
    transformer = RobustScaler()

  X_scaled1 = transformer.fit_transform(X[['month', '6~8 ride', '8~10 ride', '10~12 ride', '12~14 ride', '14~16 ride',
                                         '16~18 ride', '6~8 takeoff', '8~10 takeoff', '10~12 takeoff',
                                         '12~14 takeoff', '14~16 takeoff', '16~18 takeoff']])
  X_scaled2 = X.copy()
  X_scaled2[['month', '6~8 ride', '8~10 ride', '10~12 ride', '12~14 ride', '14~16 ride',
                                         '16~18 ride', '6~8 takeoff', '8~10 takeoff', '10~12 takeoff',
                                         '12~14 takeoff', '14~16 takeoff', '16~18 takeoff']] = X_scaled1


  X_scaled3 = transformer.transform(X_test[['month', '6~8 ride', '8~10 ride', '10~12 ride', '12~14 ride', '14~16 ride',
                                         '16~18 ride', '6~8 takeoff', '8~10 takeoff', '10~12 takeoff',
                                         '12~14 takeoff', '14~16 takeoff', '16~18 takeoff']])
  X_scaled4 = X_test.copy()
  X_scaled4[['month', '6~8 ride', '8~10 ride', '10~12 ride', '12~14 ride', '14~16 ride',
                                         '16~18 ride', '6~8 takeoff', '8~10 takeoff', '10~12 takeoff',
                                         '12~14 takeoff', '14~16 takeoff', '16~18 takeoff']] = X_scaled3

  X_scaled2[['route_ID', 'bus_station_PK','type']] = X_scaled2[['route_ID', 'bus_station_PK','type']].astype('category')
  X_scaled4[['route_ID', 'bus_station_PK','type']] = X_scaled4[['route_ID', 'bus_station_PK','type']].astype('category')
  return X_scaled2, X_scaled4

# 2. 기본 선형회귀 적용 과정

## 2-1. 원핫인코딩

In [ ]:
def lr_org(scaler, X_train, X_test, y_train, y_test):
  X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
  lr = LinearRegression()

  lr.fit(X_train ,y_train)

  #검증 데이터 적용
  y_preds_tuned1 = model.predict(X_valid)

  mse_tuned1 = mean_squared_error(y_valid, y_preds_tuned1)
  rmse1_tuned1 = np.sqrt(mse_tuned1)
  r2_tuned1 = r2_score(y_valid, y_preds_tuned1)

  print(f'{scaler} 스케일러를 적용한 경우의 평가')
  print('검증 MSE : {0:.3f}   검증 RMSE : {1:.3F}'.format(mse_tuned1 , rmse1_tuned1))
  print()
  print('검증 R2 score : {0:.3f}'.format(r2_tuned1))

  #테스트 데이터 적용
  y_preds_tuned2 = model.predict(X_test)

  mse_tuned2 = mean_squared_error(y_test, y_preds_tuned2)
  rmse1_tuned2 = np.sqrt(mse_tuned2)
  r2_tuned2 = r2_score(y_test, y_preds_tuned2)

  print('테스트 MSE : {0:.3f}   테스트 RMSE : {1:.3F}'.format(mse_tuned2 , rmse1_tuned2))
  print()
  print('테스트 R2 score : {0:.3f}'.format(r2_tuned2))

In [ ]:
lr_org('minmax', X_minmax, X_test_minmax, y, y_test)

In [ ]:
lr_org('standard', X_standard, X_test_standard, y, y_test)

In [ ]:
lr_org('robust', X_robust, X_test_robust, y, y_test)

## 2-2. GridSearch

In [ ]:
alphas = [0, 0.1, 1, 10, 100]

for alpha in alphas:
    ridge = Ridge(alpha = alpha)

    r2_scores = cross_val_score(ridge, X_data, y_target, scoring="r2", cv = 5)
    mean_r2 = np.mean(r2_scores)
    print('alpha {0} 일 때 Ridge 5 folds 의 평균 R2 : {1:.3f} '.format(alpha, mean_r2))

In [ ]:
alphas = [0, 0.1, 1, 10, 100]

for alpha in alphas:
    lasso = Lasso(alpha = alpha)

    r2_scores = cross_val_score(lasso, X_data, y_target, scoring="r2", cv = 5)
    mean_r2 = np.mean(r2_scores)
    print('alpha {0} 일 때 Lasso 5 folds 의 평균 R2 : {1:.3f} '.format(alpha, mean_r2))

In [ ]:
alphas = [0, 0.1, 1, 10, 100]

for alpha in alphas:
    elasticNet = ElasticNet(alpha = alpha)

    r2_scores = cross_val_score(elasticNet, X_data, y_target, scoring="r2", cv = 5)
    mean_r2 = np.mean(r2_scores)
    print('alpha {0} 일 때 ElasticNet 5 folds 의 평균 R2 : {1:.3f} '.format(alpha, mean_r2))

# 3. LGBM - 원핫인코딩

## 3-0. 사용함수 정의

In [ ]:
def hypertuning(X_train, y_train):
  X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
  lgb_reg_params = {
    'learning_rate':    hp.uniform('learning_rate',0.1,1),
    'max_depth':        hp.choice('max_depth',        np.arange(2, 100, 1, dtype=int)),
    'colsample_bytree': hp.uniform('colsample_bytree',0.4,1),
    'subsample':        hp.uniform('subsample', 0.6, 1),
    'num_leaves':       hp.choice('num_leaves',       np.arange(1, 200, 1, dtype=int)),
    'reg_alpha':        hp.uniform('reg_alpha',0,1),
    'reg_lambda':       hp.uniform('reg_lambda',0,1),
    'n_estimators':     hp.choice('n_estimators',        np.arange(100, 500, 10, dtype=int)), 'random_state':42
    }
  def f(params):
    lgbm = LGBMRegressor(n_jobs=-1,early_stopping_rounds=None,**params)
    score = cross_val_score(lgbm, X_train, y_train, cv=3,scoring='r2',n_jobs=-1).mean()
    return score
  #r2를 기준으로 설정
  trials = Trials()
  result = fmin(
      fn=f,                           # objective function
      space=lgb_reg_params,   # parameter space
      algo=tpe.suggest,               # surrogate algorithm
      max_evals=50,                  # no. of evaluations
      trials=trials                   # trials object that keeps track of the sample results (optional)
      )
  return result
  print(result)

In [ ]:
def tune_lgbm(X_train, X_test, y_train, y_test, scaler):
  X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
  lgbm = LGBMRegressor(n_jobs = -1, random_state=42)

  lgbm_param = { 'n_estimators' : [100, 300, 500],
              'learning_rate' : [0.01, 0.05, 0.1, 0.15],
              'max_depth' : [3, 5, 7, 9, 11],
             'colsample_bytree' : [0.8, 0.9],
              'reg_alpha':[0, 0.2, 0.5, 1],
              'reg_lambda':[0, 0.2, 0.5, 1],
              'subsample':[0.6, 0.8, 1],
              'num_leaves':[100, 200, 300], 'random_state':[42]}

  lgbm_grid = GridSearchCV(lgbm, param_grid = lgbm_param, scoring="r2", n_jobs=-1, refit=True)

  lgbm_grid.fit(X_train, y_train)

  print('◎ 최적의 Parameter : %s' % lgbm_grid.best_params_)
  #검증 데이터 적용
  y_preds_tuned1 = lgbm_grid.predict(X_valid)

  mse_tuned1 = mean_squared_error(y_valid, y_preds_tuned1)
  rmse1_tuned1 = np.sqrt(mse_tuned1)
  r2_tuned1 = r2_score(y_valid, y_preds_tuned1)
  print(f'{scaler} 스케일러를 적용한 경우의 평가')
  print('검증 MSE : {0:.3f}   검증 RMSE : {1:.3F}'.format(mse_tuned1 , rmse1_tuned1))
  print()
  print('검증 R2 score : {0:.3f}'.format(r2_tuned1))

  #테스트 데이터 적용
  y_preds_tuned2 = lgbm_grid.predict(X_test)

  mse_tuned2 = mean_squared_error(y_test, y_preds_tuned2)
  rmse1_tuned2 = np.sqrt(mse_tuned2)
  r2_tuned2 = r2_score(y_test, y_preds_tuned2)

  print('테스트 MSE : {0:.3f}   테스트 RMSE : {1:.3F}'.format(mse_tuned2 , rmse1_tuned2))
  print()
  print('테스트 R2 score : {0:.3f}'.format(r2_tuned2))

In [ ]:
def hypertuned_model(result):
  return LGBMRegressor(n_jobs=-1,
                            n_estimators=int(result['n_estimators']),
                            learning_rate=round(result['learning_rate'], 5),
                            max_depth=int(result['max_depth']),
                            colsample_bytree=round(result['colsample_bytree'], 5),
                            subsample = round(result['subsample'], 5),
                            num_leaves =int(result['num_leaves']),
                            reg_alpha = round(result['reg_alpha'], 5),
                            reg_lambda =round(result['reg_lambda'], 5), random_state=42
                           )

In [ ]:
def model_eval(model, X_train, X_test, y_train, y_test, scaler):
  X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

  model.fit(X_train, y_train)

  #검증 데이터 적용
  y_preds_tuned1 = model.predict(X_valid)

  mse_tuned1 = mean_squared_error(y_valid, y_preds_tuned1)
  rmse1_tuned1 = np.sqrt(mse_tuned1)
  r2_tuned1 = r2_score(y_valid, y_preds_tuned1)
  print(f'{scaler} 스케일러를 적용한 경우의 평가')
  print('검증 MSE : {0:.3f}   검증 RMSE : {1:.3F}'.format(mse_tuned1 , rmse1_tuned1))
  print()
  print('검증 R2 score : {0:.3f}'.format(r2_tuned1))

  #테스트 데이터 적용
  y_preds_tuned2 = model.predict(X_test)

  mse_tuned2 = mean_squared_error(y_test, y_preds_tuned2)
  rmse1_tuned2 = np.sqrt(mse_tuned2)
  r2_tuned2 = r2_score(y_test, y_preds_tuned2)

  print('테스트 MSE : {0:.3f}   테스트 RMSE : {1:.3F}'.format(mse_tuned2 , rmse1_tuned2))
  print()
  print('테스트 R2 score : {0:.3f}'.format(r2_tuned2))

## 3-1. 기본 LGBM

In [ ]:
lgbm = LGBMRegressor(n_jobs=-1, random_state=42)

In [ ]:
#원핫인코딩, MinMax, 기본 LGBM
model_eval(lgbm, X_minmax, X_test_minmax, y, y_test, 'minmax')

In [ ]:
#원핫인코딩, standard, 기본 LGBM
model_eval(lgbm, X_standard, X_test_standard, y, y_test, 'standard')

In [ ]:
#원핫인코딩, robust, 기본 LGBM
model_eval(lgbm, X_robust, X_test_robust, y, y_test, 'robust')

## 3-2. hyperopt 튜닝

In [ ]:
result1 = hypertuning(X_minmax, y)
result2 = hypertuning(X_standard, y)
result3 = hypertuning(X_robust, y)

In [ ]:
minmax_tuned = hypertuned_model(result1)
standard_tuned = hypertuned_model(result2)
robust_tuned = hypertuned_model(result3)

In [ ]:
#원핫인코딩, MinMax, hyperopt 튜닝
model_eval(minmax_tuned, X_minmax, X_test_minmax, y, y_test, 'minmax')

In [ ]:
#원핫인코딩, standard, hyperopt 튜닝
model_eval(standard_tuned, X_standard, X_test_standard, y, y_test, 'standard')

In [ ]:
#원핫인코딩, robust, hyperopt 튜닝
model_eval(robust_tuned, X_robust, X_test_robust, y, y_test, 'robust')

## 3-3. GridSearchCV

In [ ]:
#원핫인코딩, MinMax, GridSearchCV 튜닝
tune_lgbm(X_minmax, X_test_minmax, y, y_test, 'minmax')

In [ ]:
#원핫인코딩, standard, GridSearchCV 튜닝
tune_lgbm(X_standard, X_test_standard, y, y_test, 'standard')

In [ ]:
#원핫인코딩, robust, GridSearchCV 튜닝
tune_lgbm(X_robust, X_test_robust, y, y_test, 'robust')

# 4. LGBM - 카테고리형 변환

In [ ]:
X

,month,type,route_ID,6~8 ride,8~10 ride,10~12 ride,12~14 ride,14~16 ride,16~18 ride,6~8 takeoff,8~10 takeoff,10~12 takeoff,12~14 takeoff,14~16 takeoff,16~18 takeoff,bus_station_PK
0,202209,0,123000010.0,654,809,576,574,643,651,354,753,769,800,763,991,77100000001
1,202209,0,100100073.0,788,1265,1217,1074,839,731,325,782,696,982,803,950,67100000001
2,202209,0,100100549.0,227,677,282,193,381,558,708,1114,504,450,353,209,31100000002
3,202209,0,100100012.0,89,171,101,76,92,96,653,1070,287,232,159,92,58100000002
4,202209,0,100100009.0,376,759,381,312,344,464,1064,1765,509,460,451,228,24100000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450525,202308,0,100100495.0,0,0,0,0,0,0,0,1,0,0,0,1,81999800002
450526,202308,4,100100391.0,2,2,0,1,1,1,7,0,1,1,1,3,1999800003
450527,202308,4,100100391.0,0,0,0,0,0,0,1,3,1,2,5,3,60999800004
450528,202308,4,100100400.0,0,0,0,0,0,0,2,11,3,7,2,11,78999800005


In [ ]:
X_minmax2, X_test_minmax2 = cat_scale('minmax', X, X_test)
X_standard2, X_test_standard2 = cat_scale('standard', X, X_test)
X_robust2, X_test_robust2 = cat_scale('robust', X, X_test)

In [ ]:
X_minmax2

,month,type,route_ID,6~8 ride,8~10 ride,10~12 ride,12~14 ride,14~16 ride,16~18 ride,6~8 takeoff,8~10 takeoff,10~12 takeoff,12~14 takeoff,14~16 takeoff,16~18 takeoff,bus_station_PK
0,0.0,0,123000010.0,0.046206,0.022617,0.022381,0.028438,0.039701,0.027805,0.019893,0.027561,0.038782,0.059827,0.051273,0.044068,77100000001
1,0.0,0,100100073.0,0.055673,0.035365,0.047288,0.053210,0.051803,0.031222,0.018264,0.028623,0.035100,0.073437,0.053961,0.042245,67100000001
2,0.0,0,100100549.0,0.016038,0.018926,0.010957,0.009562,0.023524,0.023833,0.039786,0.040774,0.025417,0.033652,0.023722,0.009294,31100000002
3,0.0,0,100100012.0,0.006288,0.004781,0.003924,0.003765,0.005680,0.004100,0.036696,0.039164,0.014474,0.017350,0.010685,0.004091,58100000002
4,0.0,0,100100009.0,0.026565,0.021219,0.014804,0.015458,0.021240,0.019818,0.059792,0.064602,0.025669,0.034400,0.030307,0.010139,24100000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450525,1.0,0,100100495.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000037,0.000000,0.000000,0.000000,0.000044,81999800002
450526,1.0,4,100100391.0,0.000141,0.000056,0.000000,0.000050,0.000062,0.000043,0.000393,0.000000,0.000050,0.000075,0.000067,0.000133,1999800003
450527,1.0,4,100100391.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000056,0.000110,0.000050,0.000150,0.000336,0.000133,60999800004
450528,1.0,4,100100400.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000112,0.000403,0.000151,0.000523,0.000134,0.000489,78999800005


In [ ]:
X_standard2

,month,type,route_ID,6~8 ride,8~10 ride,10~12 ride,12~14 ride,14~16 ride,16~18 ride,6~8 takeoff,8~10 takeoff,10~12 takeoff,12~14 takeoff,14~16 takeoff,16~18 takeoff,bus_station_PK
0,-1.453191,0,123000010.0,0.647826,0.426289,0.572253,0.448661,0.464641,0.206509,0.120906,0.271949,0.930642,0.961547,0.818119,0.873996,77100000001
1,-1.453191,0,100100073.0,0.923283,1.069386,2.058066,1.523664,0.838093,0.320218,0.070051,0.306003,0.777586,1.353058,0.904012,0.803947,67100000001
2,-1.453191,0,100100549.0,-0.229934,0.240130,-0.109228,-0.370492,-0.034566,0.074322,0.741696,0.695862,0.375028,0.208641,-0.062276,-0.462063,31100000002
3,-1.453191,0,100100012.0,-0.513613,-0.473481,-0.528778,-0.622042,-0.585217,-0.582348,0.645246,0.644194,-0.079946,-0.260312,-0.478854,-0.661959,58100000002
4,-1.453191,0,100100009.0,0.076357,0.355774,0.120251,-0.114641,-0.105064,-0.059286,1.365993,1.460315,0.385512,0.230152,0.148160,-0.429601,24100000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450525,0.781827,0,100100495.0,-0.696566,-0.714642,-0.762892,-0.785443,-0.760511,-0.718799,-0.499883,-0.611106,-0.681685,-0.759382,-0.820275,-0.817434,81999800002
450526,0.781827,4,100100391.0,-0.692455,-0.711822,-0.762892,-0.783293,-0.758605,-0.717377,-0.487608,-0.612280,-0.679589,-0.757231,-0.818128,-0.814017,1999800003
450527,0.781827,4,100100391.0,-0.696566,-0.714642,-0.762892,-0.785443,-0.760511,-0.718799,-0.498130,-0.608757,-0.679589,-0.755079,-0.809539,-0.814017,60999800004
450528,0.781827,4,100100400.0,-0.696566,-0.714642,-0.762892,-0.785443,-0.760511,-0.718799,-0.496376,-0.599363,-0.675395,-0.744324,-0.815981,-0.800349,78999800005


In [ ]:
X_robust2

,month,type,route_ID,6~8 ride,8~10 ride,10~12 ride,12~14 ride,14~16 ride,16~18 ride,6~8 takeoff,8~10 takeoff,10~12 takeoff,12~14 takeoff,14~16 takeoff,16~18 takeoff,bus_station_PK
0,-1.000000,0,123000010.0,1.230179,0.909091,0.994667,0.833735,0.898649,0.645338,0.862069,0.931907,1.737160,1.549865,1.230024,1.290385,77100000001
1,-1.000000,0,100100073.0,1.572890,1.706294,2.704000,2.038554,1.340090,0.791590,0.750958,0.988327,1.516616,2.040431,1.326877,1.211538,67100000001
2,-1.000000,0,100100549.0,0.138107,0.678322,0.210667,-0.084337,0.308559,0.475320,2.218391,1.634241,0.936556,0.606469,0.237288,-0.213462,31100000002
3,-1.000000,0,100100012.0,-0.214834,-0.206294,-0.272000,-0.366265,-0.342342,-0.369287,2.007663,1.548638,0.280967,0.018868,-0.232446,-0.438462,58100000002
4,-1.000000,0,100100009.0,0.519182,0.821678,0.474667,0.202410,0.225225,0.303473,3.582375,2.900778,0.951662,0.633423,0.474576,-0.176923,24100000002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450525,0.053191,0,100100495.0,-0.442455,-0.505245,-0.541333,-0.549398,-0.549550,-0.544790,-0.494253,-0.531128,-0.586103,-0.606469,-0.617433,-0.613462,81999800002
450526,0.053191,4,100100391.0,-0.437340,-0.501748,-0.541333,-0.546988,-0.547297,-0.542962,-0.467433,-0.533074,-0.583082,-0.603774,-0.615012,-0.609615,1999800003
450527,0.053191,4,100100391.0,-0.442455,-0.505245,-0.541333,-0.549398,-0.549550,-0.544790,-0.490421,-0.527237,-0.583082,-0.601078,-0.605327,-0.609615,60999800004
450528,0.053191,4,100100400.0,-0.442455,-0.505245,-0.541333,-0.549398,-0.549550,-0.544790,-0.486590,-0.511673,-0.577039,-0.587601,-0.612591,-0.594231,78999800005


## 4-1. 기본 LGBM

In [ ]:
lgbm = LGBMRegressor(n_jobs=-1, random_state=42)

In [ ]:
#카테고리형 변환, MinMax, 기본 LGBM
model_eval(lgbm, X_minmax2, X_test_minmax2, y, y_test, 'minmax')

minmax 스케일러를 적용한 경우의 평가
검증 MSE : 32102.030   검증 RMSE : 179.170

검증 R2 score : 0.956
테스트 MSE : 28334.011   테스트 RMSE : 168.327

테스트 R2 score : 0.959


In [ ]:
#카테고리형 변환, standard, 기본 LGBM
model_eval(lgbm, X_standard2, X_test_standard2, y, y_test, 'standard')

standard 스케일러를 적용한 경우의 평가
검증 MSE : 32307.147   검증 RMSE : 179.742

검증 R2 score : 0.955
테스트 MSE : 28500.714   테스트 RMSE : 168.822

테스트 R2 score : 0.959


In [ ]:
#카테고리형 변환, robust, 기본 LGBM
model_eval(lgbm, X_robust2, X_test_robust2, y, y_test, 'robust')

robust 스케일러를 적용한 경우의 평가
검증 MSE : 32752.750   검증 RMSE : 180.977

검증 R2 score : 0.955
테스트 MSE : 28658.803   테스트 RMSE : 169.289

테스트 R2 score : 0.958


## 4-2. hyperopt 튜닝

In [ ]:
result1 = hypertuning(X_minmax2, y)
result2 = hypertuning(X_standard2, y)
result3 = hypertuning(X_robust2, y)

100%|██████████| 50/50 [21:25<00:00, 25.70s/trial, best loss: 0.8757064779035152]


In [ ]:
minmax_tuned = hypertuned_model(result1)
standard_tuned = hypertuned_model(result2)
robust_tuned = hypertuned_model(result3)

In [ ]:
#카테고리형 변환, MinMax, hyperopt 튜닝
model_eval(minmax_tuned, X_minmax2, X_test_minmax2, y, y_test, 'minmax')

minmax 스케일러를 적용한 경우의 평가
검증 MSE : 76066.953   검증 RMSE : 275.802

검증 R2 score : 0.895
테스트 MSE : 70486.795   테스트 RMSE : 265.493

테스트 R2 score : 0.898


In [ ]:
#카테고리형 변환, standard, hyperopt 튜닝
model_eval(standard_tuned, X_standard2, X_test_standard2, y, y_test, 'standard')

standard 스케일러를 적용한 경우의 평가
검증 MSE : 65084.125   검증 RMSE : 255.116

검증 R2 score : 0.910
테스트 MSE : 62857.909   테스트 RMSE : 250.715

테스트 R2 score : 0.909


In [ ]:
#카테고리형 변환, robust, hyperopt 튜닝
model_eval(robust_tuned, X_robust2, X_test_robust2, y, y_test, 'robust')

robust 스케일러를 적용한 경우의 평가
검증 MSE : 67190.138   검증 RMSE : 259.211

검증 R2 score : 0.907
테스트 MSE : 63469.158   테스트 RMSE : 251.931

테스트 R2 score : 0.908


In [ ]:
minmax = minmax_tuned.predict(X_test_minmax2)
standard = standard_tuned.predict(X_test_standard2)
robust = robust_tuned.predict(X_test_robust2)

In [ ]:
type(robust)
type(y_test)

pandas.core.series.Series

In [ ]:
submission = pd.DataFrame({'id': test['bus_station_PK'],
                           '18~20_ride': y_test, 'minmax':minmax, 'standard':standard, 'robust':robust})
submission

,id,18~20_ride,minmax,standard,robust
0,66100000001,720,763.388442,607.065600,664.780032
1,75100000001,867,748.998296,1003.231004,709.439559
2,31100000002,365,591.280746,473.299033,227.233108
3,24100000002,523,655.540669,419.955137,483.188389
4,12100000002,600,849.418156,582.347780,565.338143
...,...,...,...,...,...
113141,81999800002,0,28.676590,24.885587,39.301294
113142,1999800003,4,4.345891,11.596533,1.125851
113143,60999800004,0,4.345891,11.596533,1.125851
113144,127999800005,0,4.345891,29.630701,21.362943


In [ ]:
# submission.csv 파일로 저장
submission.to_csv('submission.csv', index=False)

## 4-3. GridSearchCV

In [ ]:
#카테고리형 변환, MinMax, GridSearchCV 튜닝
tune_lgbm(X_minmax2, X_test_minmax2, y, y_test, 'minmax')

In [ ]:
#카테고리형 변환, standard, GridSearchCV 튜닝
tune_lgbm(X_standard2, X_test_standard2, y, y_test, 'standard')

In [ ]:
#카테고리형 변환, robust, GridSearchCV 튜닝
tune_lgbm(X_robust2, X_test_robust2, y, y_test, 'robust')